In [5]:
import pandas as pd
import operator
from model.model import Model
import torch
from sklearn.metrics import r2_score
from preprocess import normalize
import torch.nn.functional as F
data = pd.read_csv('utils/df_imputed.csv', index_col=0)
data = data.drop(columns='date')

In [6]:
raw = pd.read_csv('../data/df_20210510.csv', index_col=0)['GPP_NT_VUT_REF']
raw = raw[raw.index != 'CN-Cng']
sites = raw.index.unique()

masks = []
for s in sites:
    mask = raw[raw.index == s].isna().values
    masks.append(list(map(operator.not_, mask)))

In [7]:
meta_columns=['classid','igbp_land_use']
sites = data.index.unique()
meta_data = pd.get_dummies(data[meta_columns])
sensor_data = data.drop(columns=['classid', 'plant_functional_type', 'koeppen_code','igbp_land_use', 'GPP_NT_VUT_REF'])

# Batch by site
df_sensor = [normalize(sensor_data[sensor_data.index == site]) for site in sites if sensor_data[sensor_data.index == site].size != 0 ]
df_meta = [meta_data[meta_data.index == site] for site in sites if meta_data[meta_data.index == site].size != 0]
df_gpp = [data[data.index == site]['GPP_NT_VUT_REF'] for site in sites if data[data.index == site].size != 0]   
df_gpp = [(df_gpp[i]-df_gpp[i].mean())/df_gpp[i].std() for i in range(len(df_gpp))]

In [8]:
classid = ['DBF', 'ENF', 'GRA', 'MF'] 

In [9]:
#Split dataframes by veg type
dbf_sites = []
enf_sites = []
gra_sites = []
mf_sites  = []
for i in range(len(df_sensor)):
    if df_meta[i]['classid_DBF'][0] == 1:
        dbf_sites.append(i)
    elif df_meta[i]['classid_ENF'][0] == 1:
        enf_sites.append(i)
    elif df_meta[i]['classid_GRA'][0] == 1:
        gra_sites.append(i)
    elif df_meta[i]['classid_MF'][0] == 1:
        mf_sites.append(i)

In [34]:
# test_sites_in.append(train_sites[0])
# train_sites.remove(train_sites[0])
# test_sites_in.append(train_sites[-1])
# train_sites.remove(train_sites[-1])

In [15]:
#define model parameter
DEVICE = "cuda:7"
INPUT_FEATURES = len(df_sensor[0].columns) 
HIDDEN_DIM = 256
CONDITIONAL_FEATURES = len(df_meta[0].columns)

In [19]:
from copy import deepcopy

In [20]:
# train_sites = deepcopy(dbf_sites)

In [22]:
# train_sites

[23, 26, 30, 34, 43, 45, 50, 51]

In [ ]:
def validate(x_test,y_test,conditional_test, sites):
    i = 0
    r2 = 0
    for (x, y,conditional) in zip(x_test, y_test, conditional_test):
        x = torch.FloatTensor(x).to(DEVICE)
        y = torch.FloatTensor(y).to(DEVICE)
        c = torch.FloatTensor(conditional).to(DEVICE)
        y_pred = model(x, c)
        test_loss = F.mse_loss( y_pred, y)
        r2 += r2_score(y_true=y.detach().cpu().numpy()[masks[sites[i]]], y_pred=y_pred.detach().cpu().numpy()[masks[sites[i]]])
        i += 1
    r2 /= i
    return r2

In [ ]:
def compute_bias(x_test,y_test,conditional_test, sites):
    i = 0
    bias = []
    for (x, y,conditional) in zip(x_test, y_test, conditional_test):
        x = torch.FloatTensor(x).to(DEVICE)
        y = torch.FloatTensor(y).to(DEVICE)
        c = torch.FloatTensor(conditional).to(DEVICE)
        y_pred = model(x, c)
        bias.append((y_pred - y).detach().cpu().numpy())
        i += 1
    
    return np.concatenate(bias)

In [59]:
from tqdm import tqdm

In [60]:
# This cell computes the bias by vegetation types. 
# We divided the training one into two parts, namely train_sites and test_sites_in by leave-one-out method.
# We also test on all other types that are not used for training. 
# Need to note the dataframe in outter loop also in need of changing when one want to train on other vegetation types. 

all_bias_gra = []
all_bias_enf = []
all_bias_dbf = []
all_bias_mf = []

for k in tqdm(range(len(dbf_sites))):
    
    # define model
    model = Model(INPUT_FEATURES, CONDITIONAL_FEATURES, HIDDEN_DIM,0, 1).to(DEVICE)
    optimizer = torch.optim.Adam(model.parameters())
    
    # define data 
    train_sites = deepcopy(dbf_sites)
    train_sites.remove(dbf_sites[k])
    test_sites_in = [dbf_sites[k]]
    
    x_train = [df_sensor[i].values for i in train_sites]
    y_train = [df_gpp[i].values.reshape(-1,1) for i in train_sites]
    conditional_train = [df_meta[i].values for i in train_sites]

    x_test_gra = [df_sensor[i].values for i in gra_sites]
    y_test_gra = [df_gpp[i].values.reshape(-1,1) for i in gra_sites]
    conditional_test_gra = [df_meta[i].values for i in gra_sites]

    x_test_enf = [df_sensor[i].values for i in enf_sites]
    y_test_enf = [df_gpp[i].values.reshape(-1,1) for i in enf_sites]
    conditional_test_enf = [df_meta[i].values for i in enf_sites]

    x_test_mf = [df_sensor[i].values for i in mf_sites]
    y_test_mf = [df_gpp[i].values.reshape(-1,1) for i in mf_sites]
    conditional_test_mf = [df_meta[i].values for i in mf_sites]


    x_test_dbf = [df_sensor[i].values for i in test_sites_in]
    y_test_dbf = [df_gpp[i].values.reshape(-1,1) for i in test_sites_in]
    conditional_test_dbf = [df_meta[i].values for i in test_sites_in]


    best_r2 = 0
    for epoch in range(25):
        train_loss = 0.0
        train_r2 = 0.0
        model.train()
        for (x, y, conditional) in zip(x_train, y_train, conditional_train):
            x = torch.FloatTensor(x).to(DEVICE)
            y = torch.FloatTensor(y).to(DEVICE)
            c = torch.FloatTensor(conditional).to(DEVICE)
            y_pred = model(x, c)
            optimizer.zero_grad()
            loss = F.mse_loss( y_pred, y)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
            train_r2 += r2_score(y_true=y.detach().cpu().numpy(), y_pred=y_pred.detach().cpu().numpy())
        
        model.eval()
        with torch.no_grad():
                r2_gra = validate(x_test_gra, y_test_gra, conditional_test_gra, gra_sites)
                r2_enf = validate(x_test_enf, y_test_enf, conditional_test_enf, enf_sites)
                r2_mf = validate(x_test_mf, y_test_mf, conditional_test_mf, mf_sites)
                r2_dbf = validate(x_test_dbf, y_test_dbf, conditional_test_dbf, test_sites_in)
                if r2_dbf > best_r2:
                    best_r2 = r2_dbf
                    bias_gra = compute_bias(x_test_gra, y_test_gra, conditional_test_gra, gra_sites)
                    bias_enf = compute_bias(x_test_enf, y_test_enf, conditional_test_enf, enf_sites)
                    bias_mf = compute_bias(x_test_mf, y_test_mf, conditional_test_mf, mf_sites)
                    bias_dbf = compute_bias(x_test_dbf, y_test_dbf, conditional_test_dbf, test_sites_in)        
    all_bias_dbf.append(bias_dbf)
    all_bias_enf.append(bias_enf)
    all_bias_mf.append(bias_mf)
    all_bias_gra.append(bias_gra)

 11%|█         | 1/9 [03:37<28:56, 217.11s/it]

In [42]:
import numpy as np
np.concatenate(bias_gra).shape

(34335, 1)

In [51]:
all_bias_gra[1]

array([[-0.6882744 ],
       [-0.5787851 ],
       [-0.99682605],
       ...,
       [-0.02996317],
       [ 0.10080877],
       [ 0.26739553]], dtype=float32)